<a href="https://colab.research.google.com/github/robinsingh2022/NorthwesternMSDS422/blob/main/Assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Ingest

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import tensorflow as tf

In [ ]:
path='/content/drive/MyDrive/Assignment8/train.csv'
train= pd.read_csv(path)

In [ ]:
path='/content/drive/MyDrive/Assignment8/test.csv'
test=pd.read_csv(path)

###EDA

In [ ]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [ ]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
train[train["target"] == 0]["text"].values[1]

'I love fruits'

In [ ]:
train[train["target"] == 0]["text"].values[2]

'Summer is lovely'

In [ ]:
train[train["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [ ]:
train[train["target"] == 1]["text"].values[2]

"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"

In [ ]:
countVectorizer = feature_extraction.text.CountVectorizer()

In [ ]:
trainVectors = countVectorizer.fit_transform(train["text"])

In [ ]:
print(trainVectors[0].todense().shape)
print(trainVectors.todense())

(1, 21637)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
linModel = linear_model.RidgeClassifier()

In [ ]:
scores = model_selection.cross_val_score(linModel, trainVectors, train["target"], cv=3, scoring="f1")
scores

array([0.59453669, 0.56498283, 0.64082434])

In [ ]:
linModel.fit(trainVectors, train["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [ ]:
testVectors = countVectorizer.transform(test["text"])
linModel.predict(testVectors)

array([0, 1, 1, ..., 1, 1, 0])

In [ ]:
type(trainVectors)

scipy.sparse.csr.csr_matrix

In [ ]:
trainVectors[1].shape

(1, 21637)

In [ ]:
trainVectors.shape[0]

7613

In [ ]:
print(trainVectors[111].todense())

[[0 0 0 ... 0 0 0]]


In [ ]:
len(trainVectors.todense())

7613

In [ ]:
trainVectors[2].shape

(1, 21637)

In [ ]:
trainVectors[1]

<1x21637 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM, Dense,Dropout
from keras.initializers import Constant
from keras.optimizers import Adam



In [ ]:
model=Sequential()

model.add(Embedding(trainVectors.shape[0],32,input_length=trainVectors.shape[1]))
model.add(LSTM(64,dropout=0.1))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
optimizer=Adam(learning_rate=3e-4)


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 21637, 32)         243616    
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 268,513
Trainable params: 268,513
Non-trainable params: 0
_________________________________________________________________


In [ ]:
labels=np.array(train['target'])

In [ ]:
labels.shape

(7613,)

In [ ]:
#Technique help from CodeBasics on YouTube
text2vecmodel="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [ ]:
import tensorflow_hub as hub

In [ ]:
hublayer=hub.KerasLayer(text2vecmodel,input_shape=[],dtype=tf.string,trainable=True)

In [ ]:
trainArray=np.array(train['text'])

In [ ]:
hublayer(trainArray).shape

TensorShape([7613, 20])

In [ ]:
hubTrain=hublayer(trainArray)

In [ ]:
hubTrain.shape

TensorShape([7613, 20])

In [ ]:
model=Sequential()

model.add(Embedding(hubTrain.shape[0],32,input_length=hubTrain.shape[1]))
model.add(LSTM(64,dropout=0.1))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
optimizer=Adam(learning_rate=3e-4)


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 32)            243616    
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 268,513
Trainable params: 268,513
Non-trainable params: 0
_________________________________________________________________


In [ ]:
labels.shape

(7613,)

In [ ]:
for i in hubTrain[:5]:
  print(i)

tf.Tensor(
[ 0.00723878  0.15689617  1.2953067  -0.04077986 -0.16036902 -1.2004429
 -0.40451345 -0.5239726  -0.23558664 -0.35022184 -0.28988177 -0.47197863
 -0.07621863  0.17048942 -0.8323773   0.02260084  0.16125458 -0.64086777
 -0.569074    0.04000134], shape=(20,), dtype=float32)
tf.Tensor(
[-0.49837342  1.2828879  -0.18836126 -0.9473474   0.88058084 -0.17885518
 -0.01543167  0.38327044 -1.508848    1.7532812  -0.75493276  0.601241
 -0.14378418 -0.891944    0.43213665  1.346741   -1.4859254   1.5762157
 -0.37878057  0.9423684 ], shape=(20,), dtype=float32)
tf.Tensor(
[ 0.08814443 -0.31140247  0.6044149   1.7130805   1.3203121  -3.201303
  0.8445131   0.9621795  -1.5353138   2.2069528  -0.2790253   0.01095965
 -2.2282734   0.2043116  -2.17501     1.3441613   0.11884003 -1.2388006
 -0.25501075  0.69527215], shape=(20,), dtype=float32)
tf.Tensor(
[-1.0832024   0.6943499  -0.2933439   0.30142915  1.8872913  -2.8647776
 -0.01886398  0.17613813 -2.3882043   1.0208809   0.14414711  0.80232

In [ ]:
hubTrain=hubTrain.numpy()

In [ ]:
hubTrain.shape

(7613, 20)

In [ ]:
hubTrain

array([[ 0.00723878,  0.15689617,  1.2953067 , ..., -0.64086777,
        -0.569074  ,  0.04000134],
       [-0.49837342,  1.2828879 , -0.18836126, ...,  1.5762157 ,
        -0.37878057,  0.9423684 ],
       [ 0.08814443, -0.31140247,  0.6044149 , ..., -1.2388006 ,
        -0.25501075,  0.69527215],
       ...,
       [-1.0302866 ,  0.20015353, -0.69170207, ...,  0.5923196 ,
         1.1341598 ,  0.18402041],
       [ 0.19233367, -0.517235  , -0.7621139 , ..., -0.5327093 ,
        -1.1736155 ,  0.00943424],
       [-1.2777979 ,  0.16780092,  0.72087336, ...,  1.0953661 ,
         0.92622113,  0.41969982]], dtype=float32)

In [ ]:
model=Sequential()

model.add(Embedding(hubTrain.shape[0],32,input_length=hubTrain.shape[1]))
model.add(LSTM(64,dropout=0.1))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
optimizer=Adam(learning_rate=3e-4)


In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 32)            243616    
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 268,513
Trainable params: 268,513
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model = tf.keras.Sequential()
model.add(hublayer)
model.add(Embedding(hubTrain.shape[0],32,input_length=hubTrain.shape[1]))
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
embedding_9 (Embedding)      (None, 20, 32)            243616    
_________________________________________________________________
dense_10 (Dense)             (None, 20, 16)            528       
_________________________________________________________________
dense_11 (Dense)             (None, 20, 1)             17        
Total params: 644,181
Trainable params: 644,181
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

###Models

In [ ]:
trainArray

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [ ]:
hubTrain[hubTrain<0] = 0

In [ ]:
hubTrain.shape

(7613, 20)

In [ ]:
hubTrain

array([[0.00723878, 0.15689617, 1.2953067 , ..., 0.        , 0.        ,
        0.04000134],
       [0.        , 1.2828879 , 0.        , ..., 1.5762157 , 0.        ,
        0.9423684 ],
       [0.08814443, 0.        , 0.6044149 , ..., 0.        , 0.        ,
        0.69527215],
       ...,
       [0.        , 0.20015353, 0.        , ..., 0.5923196 , 1.1341598 ,
        0.18402041],
       [0.19233367, 0.        , 0.        , ..., 0.        , 0.        ,
        0.00943424],
       [0.        , 0.16780092, 0.72087336, ..., 1.0953661 , 0.92622113,
        0.41969982]], dtype=float32)

In [ ]:
model=Sequential()

model.add(Embedding(hubTrain.shape[0],32,input_length=hubTrain.shape[1]))
model.add(LSTM(64,dropout=0.1))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
optimizer=Adam(learning_rate=3e-4)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
fitting=model.fit(hubTrain,labels,epochs=20)

Epoch 1/20
238/238 [==============================] - 8s 25ms/step - loss: 0.6852 - accuracy: 0.5610
Epoch 2/20
238/238 [==============================] - 6s 26ms/step - loss: 0.6850 - accuracy: 0.5654
Epoch 3/20
238/238 [==============================] - 6s 25ms/step - loss: 0.6820 - accuracy: 0.5736
Epoch 4/20
238/238 [==============================] - 6s 25ms/step - loss: 0.6839 - accuracy: 0.5617
Epoch 5/20
238/238 [==============================] - 6s 25ms/step - loss: 0.6821 - accuracy: 0.5661
Epoch 6/20
238/238 [==============================] - 6s 25ms/step - loss: 0.6780 - accuracy: 0.5843
Epoch 7/20
238/238 [==============================] - 6s 25ms/step - loss: 0.6742 - accuracy: 0.5867
Epoch 8/20
238/238 [==============================] - 6s 25ms/step - loss: 0.6742 - accuracy: 0.5804
Epoch 9/20
238/238 [==============================] - 6s 25ms/step - loss: 0.6695 - accuracy: 0.5835
Epoch 10/20
238/238 [==============================] - 6s 25ms/step - loss: 0.6496 - accura

In [ ]:
model=Sequential()

model.add(Embedding(hubTrain.shape[0],32,input_length=hubTrain.shape[1]))
model.add(LSTM(32,dropout=0.1))

model.add(Dense(1, activation='sigmoid'))


In [ ]:
optimizer=Adam(learning_rate=0.0001)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
fitting=model.fit(hubTrain,labels,epochs=20)

Epoch 1/20
238/238 [==============================] - 7s 21ms/step - loss: 0.6881 - accuracy: 0.5720
Epoch 2/20
238/238 [==============================] - 5s 22ms/step - loss: 0.6826 - accuracy: 0.5749
Epoch 3/20
238/238 [==============================] - 5s 22ms/step - loss: 0.6833 - accuracy: 0.5701
Epoch 4/20
238/238 [==============================] - 5s 22ms/step - loss: 0.6833 - accuracy: 0.5695
Epoch 5/20
238/238 [==============================] - 5s 22ms/step - loss: 0.6850 - accuracy: 0.5642
Epoch 6/20
238/238 [==============================] - 5s 21ms/step - loss: 0.6820 - accuracy: 0.5736
Epoch 7/20
238/238 [==============================] - 5s 22ms/step - loss: 0.6823 - accuracy: 0.5719
Epoch 8/20
238/238 [==============================] - 5s 22ms/step - loss: 0.6834 - accuracy: 0.5668
Epoch 9/20
238/238 [==============================] - 5s 22ms/step - loss: 0.6826 - accuracy: 0.5700
Epoch 10/20
238/238 [==============================] - 5s 22ms/step - loss: 0.6803 - accura

In [ ]:
model=Sequential()

model.add(Embedding(hubTrain.shape[0],32,input_length=hubTrain.shape[1]))
model.add(LSTM(128,dropout=0.1))

model.add(Dense(1, activation='sigmoid'))
optimizer=Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
fitting=model.fit(hubTrain,labels,epochs=20)

Epoch 1/20
238/238 [==============================] - 9s 29ms/step - loss: 0.6866 - accuracy: 0.5718
Epoch 2/20
238/238 [==============================] - 7s 29ms/step - loss: 0.6834 - accuracy: 0.5716
Epoch 3/20
238/238 [==============================] - 7s 29ms/step - loss: 0.6825 - accuracy: 0.5744
Epoch 4/20
238/238 [==============================] - 7s 29ms/step - loss: 0.6819 - accuracy: 0.5750
Epoch 5/20
238/238 [==============================] - 8s 32ms/step - loss: 0.6841 - accuracy: 0.5663
Epoch 6/20
238/238 [==============================] - 7s 29ms/step - loss: 0.6800 - accuracy: 0.5792
Epoch 7/20
238/238 [==============================] - 7s 29ms/step - loss: 0.6849 - accuracy: 0.5600
Epoch 8/20
238/238 [==============================] - 7s 29ms/step - loss: 0.6831 - accuracy: 0.5650
Epoch 9/20
238/238 [==============================] - 7s 29ms/step - loss: 0.6850 - accuracy: 0.5586
Epoch 10/20
238/238 [==============================] - 7s 29ms/step - loss: 0.6787 - accura

In [ ]:
trainArray

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [ ]:
hubTrain=np.array(hublayer(trainArray))

In [ ]:
hubTrain

array([[ 0.00723878,  0.15689617,  1.2953067 , ..., -0.64086777,
        -0.569074  ,  0.04000134],
       [-0.49837342,  1.2828879 , -0.18836126, ...,  1.5762157 ,
        -0.37878057,  0.9423684 ],
       [ 0.08814443, -0.31140247,  0.6044149 , ..., -1.2388006 ,
        -0.25501075,  0.69527215],
       ...,
       [-1.0302866 ,  0.20015353, -0.69170207, ...,  0.5923196 ,
         1.1341598 ,  0.18402041],
       [ 0.19233367, -0.517235  , -0.7621139 , ..., -0.5327093 ,
        -1.1736155 ,  0.00943424],
       [-1.2777979 ,  0.16780092,  0.72087336, ...,  1.0953661 ,
         0.92622113,  0.41969982]], dtype=float32)

In [ ]:
hubTrain=np.absolute(hubTrain)

In [ ]:
model=Sequential()

model.add(Embedding(hubTrain.shape[0],32,input_length=hubTrain.shape[1]))
model.add(LSTM(64,dropout=0.1))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
optimizer=Adam(learning_rate=3e-4)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
fitting=model.fit(hubTrain,labels,epochs=20)

Epoch 1/20
238/238 [==============================] - 6s 17ms/step - loss: 0.6863 - accuracy: 0.5567
Epoch 2/20
238/238 [==============================] - 4s 17ms/step - loss: 0.6773 - accuracy: 0.5476
Epoch 3/20
238/238 [==============================] - 4s 17ms/step - loss: 0.6668 - accuracy: 0.5737
Epoch 4/20
238/238 [==============================] - 4s 17ms/step - loss: 0.6639 - accuracy: 0.5851
Epoch 5/20
238/238 [==============================] - 4s 17ms/step - loss: 0.6656 - accuracy: 0.5725
Epoch 6/20
238/238 [==============================] - 4s 17ms/step - loss: 0.6695 - accuracy: 0.5804
Epoch 7/20
238/238 [==============================] - 4s 17ms/step - loss: 0.6604 - accuracy: 0.5885
Epoch 8/20
238/238 [==============================] - 4s 18ms/step - loss: 0.6610 - accuracy: 0.5902
Epoch 9/20
238/238 [==============================] - 4s 18ms/step - loss: 0.6655 - accuracy: 0.5771
Epoch 10/20
238/238 [==============================] - 4s 18ms/step - loss: 0.6612 - accura

####Better Models

In [ ]:
df=train['text']

In [ ]:
from collections import Counter

In [ ]:
count=Counter()
for i in df.values:
  for word in i.split():
    count[word]+=1
    

In [ ]:
count

Counter({'Our': 25,
         'Deeds': 1,
         'are': 345,
         'the': 2575,
         'Reason': 5,
         'of': 1722,
         'this': 335,
         '#earthquake': 8,
         'May': 16,
         'ALLAH': 1,
         'Forgive': 1,
         'us': 53,
         'all': 178,
         'Forest': 29,
         'fire': 148,
         'near': 50,
         'La': 11,
         'Ronge': 1,
         'Sask.': 1,
         'Canada': 4,
         'All': 52,
         'residents': 4,
         'asked': 9,
         'to': 1805,
         "'shelter": 1,
         'in': 1757,
         "place'": 1,
         'being': 67,
         'notified': 1,
         'by': 469,
         'officers.': 1,
         'No': 53,
         'other': 31,
         'evacuation': 32,
         'or': 181,
         'shelter': 4,
         'place': 22,
         'orders': 11,
         'expected': 11,
         '13,000': 1,
         'people': 146,
         'receive': 2,
         '#wildfires': 5,
         'California': 89,
         'Just': 48,
  

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer=Tokenizer(num_words=len(count))

In [ ]:
tokenizer.fit_on_texts(count)

In [ ]:
index=tokenizer.word_index
type(index)

dict

In [ ]:
sequences=tokenizer.texts_to_sequences(df)

In [ ]:
maxLength = max(len(x) for x in sequences )

In [ ]:
maxLength

33

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
paddedDF=pad_sequences(sequences,maxlen=35,padding='post',truncating='post')

In [ ]:
paddedDF[1]

array([ 280,   53,  772,  361, 4535, 4536,  362,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0], dtype=int32)

In [ ]:
paddedDF.shape

(7613, 35)

In [ ]:
labels=train['target']

In [ ]:
labels

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM, Dense,Dropout
from keras.initializers import Constant
from keras.optimizers import Adam


In [ ]:

model=Sequential()

model.add(Embedding(32700,32,input_length=paddedDF.shape[1]))
model.add(LSTM(64,dropout=0.1))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
optimizer=Adam(learning_rate=0.05)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
index2=list(index.items())
index2[-1]

('ymy4rskq3d', 22700)

In [ ]:
fitting=model.fit(paddedDF,labels,epochs=20)

Epoch 1/20
238/238 [==============================] - 12s 41ms/step - loss: 0.7166 - accuracy: 0.5358
Epoch 2/20
238/238 [==============================] - 10s 40ms/step - loss: 0.6862 - accuracy: 0.5641
Epoch 3/20
238/238 [==============================] - 9s 40ms/step - loss: 0.6887 - accuracy: 0.5590
Epoch 4/20
238/238 [==============================] - 9s 39ms/step - loss: 0.6910 - accuracy: 0.5477
Epoch 5/20
238/238 [==============================] - 9s 39ms/step - loss: 0.6839 - accuracy: 0.5665
Epoch 6/20
238/238 [==============================] - 10s 40ms/step - loss: 0.6896 - accuracy: 0.5542
Epoch 7/20
238/238 [==============================] - 10s 40ms/step - loss: 0.6909 - accuracy: 0.5509
Epoch 8/20
238/238 [==============================] - 10s 41ms/step - loss: 0.6888 - accuracy: 0.5606
Epoch 9/20
238/238 [==============================] - 10s 41ms/step - loss: 0.6865 - accuracy: 0.5708
Epoch 10/20
238/238 [==============================] - 10s 41ms/step - loss: 0.6905 -

In [ ]:
testtext=test['text']

In [ ]:
testtext

0                      Just happened a terrible car crash
1       Heard about #earthquake is different cities, s...
2       there is a forest fire at spot pond, geese are...
3                Apocalypse lighting. #Spokane #wildfires
4           Typhoon Soudelor kills 28 in China and Taiwan
                              ...                        
3258    EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259    Storm in RI worse than last hurricane. My city...
3260    Green Line derailment in Chicago http://t.co/U...
3261    MEG issues Hazardous Weather Outlook (HWO) htt...
3262    #CityofCalgary has activated its Municipal Eme...
Name: text, Length: 3263, dtype: object

In [ ]:
count2=Counter()
for i in testtext.values:
  for word in i.split():
    count2[word]+=1

In [ ]:
tokenizer=Tokenizer(num_words=len(count2))

In [ ]:
tokenizer.fit_on_texts(count2)

In [ ]:
index2=tokenizer.word_index


In [ ]:
index2

{'t': 1,
 'co': 2,
 'http': 3,
 'https': 4,
 "'": 5,
 'amp': 6,
 '1': 7,
 '3': 8,
 '2': 9,
 'up': 10,
 'news': 11,
 'gt': 12,
 'a': 13,
 'i': 14,
 '7': 15,
 'm': 16,
 '5': 17,
 'buildings': 18,
 'it': 19,
 '10': 20,
 'w': 21,
 'is': 22,
 'the': 23,
 'in': 24,
 '00': 25,
 '8': 26,
 'fire': 27,
 'you': 28,
 'out': 29,
 'p': 30,
 '11': 31,
 'no': 32,
 'me': 33,
 'day': 34,
 '2015': 35,
 'on': 36,
 'now': 37,
 's': 38,
 '9': 39,
 'that': 40,
 'year': 41,
 'to': 42,
 '4': 43,
 'o': 44,
 'bomb': 45,
 'disaster': 46,
 '16': 47,
 'old': 48,
 'video': 49,
 'night': 50,
 '15': 51,
 'b': 52,
 'earthquake': 53,
 'them': 54,
 'ablaze': 55,
 'c': 56,
 'new': 57,
 'lol': 58,
 'dead': 59,
 'time': 60,
 'food': 61,
 'fires': 62,
 'ok': 63,
 'wow': 64,
 'plan': 65,
 'terrorist': 66,
 'life': 67,
 'do': 68,
 'for': 69,
 'not': 70,
 'people': 71,
 'shit': 72,
 'this': 73,
 'go': 74,
 'hiroshima': 75,
 'today': 76,
 'god': 77,
 'rt': 78,
 'services': 79,
 '0': 80,
 'nuclear': 81,
 'report': 82,
 'post': 83

In [ ]:
testsequences=tokenizer.texts_to_sequences(testtext)

In [ ]:
maxLength = max(len(x) for x in testsequences )

In [ ]:
maxLength

31

In [ ]:
paddedTest=pad_sequences(testsequences,maxlen=35,padding='post',truncating='post')

In [ ]:
paddedTest.shape

(3263, 35)

In [ ]:
model.predict(paddedTest)

array([[0.17765188],
       [0.9387053 ],
       [0.03059304],
       ...,
       [0.8738878 ],
       [0.88976127],
       [0.8245255 ]], dtype=float32)

In [ ]:
prediction=model.predict(paddedTest)

In [ ]:
lis=[]
for i in prediction:
  if i>=0.5:
    i=1
    lis.append(i)
  else:
    i=0
    lis.append(i)

In [ ]:
len(lis)

3263

In [ ]:
submissionAssignment8 = pd.DataFrame({'id': list(range(1, len(lis)+1)), 'label': lis})
submissionAssignment8.to_csv('submissionAssignment8.csv', index=False)

Kaggle score = 0.54030

In [ ]:

model=Sequential()

model.add(Embedding(32700,32,input_length=paddedDF.shape[1]))
model.add(LSTM(64,dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
optimizer=Adam(learning_rate=3e-4)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
fitting=model.fit(paddedDF,labels,epochs=20)

Epoch 1/20
238/238 [==============================] - 11s 40ms/step - loss: 0.6707 - accuracy: 0.5775
Epoch 2/20
238/238 [==============================] - 10s 40ms/step - loss: 0.3899 - accuracy: 0.8330
Epoch 3/20
238/238 [==============================] - 10s 40ms/step - loss: 0.2460 - accuracy: 0.9122
Epoch 4/20
238/238 [==============================] - 10s 40ms/step - loss: 0.1503 - accuracy: 0.9480
Epoch 5/20
238/238 [==============================] - 10s 40ms/step - loss: 0.1017 - accuracy: 0.9674
Epoch 6/20
238/238 [==============================] - 10s 40ms/step - loss: 0.0708 - accuracy: 0.9782
Epoch 7/20
238/238 [==============================] - 9s 39ms/step - loss: 0.0410 - accuracy: 0.9862
Epoch 8/20
238/238 [==============================] - 10s 40ms/step - loss: 0.0318 - accuracy: 0.9922
Epoch 9/20
238/238 [==============================] - 9s 40ms/step - loss: 0.0400 - accuracy: 0.9885
Epoch 10/20
238/238 [==============================] - 10s 40ms/step - loss: 0.0271 

In [ ]:
prediction=model.predict(paddedTest)

In [ ]:
lis=[]
for i in prediction:
  if i>=0.5:
    i=1
    lis.append(i)
  else:
    i=0
    lis.append(i)

In [ ]:
submissionAssignment8_3 = pd.DataFrame({'id': list(range(1, len(lis)+1)), 'label': lis})
submissionAssignment8_3.to_csv('submissionAssignment8_3.csv', index=False)

Kaggle Score = 0.50291

In [ ]:

model=Sequential()

model.add(Embedding(32700,32,input_length=paddedDF.shape[1]))
model.add(LSTM(128,dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
optimizer=Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
fitting=model.fit(paddedDF,labels,epochs=20)

Epoch 1/20
238/238 [==============================] - 15s 56ms/step - loss: 0.7157 - accuracy: 0.5299
Epoch 2/20
238/238 [==============================] - 13s 56ms/step - loss: 0.7328 - accuracy: 0.5365
Epoch 3/20
238/238 [==============================] - 13s 55ms/step - loss: 0.6578 - accuracy: 0.6314
Epoch 4/20
238/238 [==============================] - 13s 56ms/step - loss: 0.5808 - accuracy: 0.7046
Epoch 5/20
238/238 [==============================] - 13s 56ms/step - loss: 0.5070 - accuracy: 0.7580
Epoch 6/20
238/238 [==============================] - 13s 56ms/step - loss: 0.4570 - accuracy: 0.7899
Epoch 7/20
238/238 [==============================] - 13s 56ms/step - loss: 0.4463 - accuracy: 0.7983
Epoch 8/20
238/238 [==============================] - 14s 57ms/step - loss: 0.4208 - accuracy: 0.8124
Epoch 9/20
238/238 [==============================] - 14s 57ms/step - loss: 0.3953 - accuracy: 0.8299
Epoch 10/20
238/238 [==============================] - 13s 57ms/step - loss: 0.396

In [ ]:
prediction2=model.predict(paddedTest)

In [ ]:
lis2=[]
for i in prediction:
  if i>=0.5:
    i=1
    lis.append(i)
  else:
    i=0
    lis.append(i)

In [ ]:
submissionAssignment8_2 = pd.DataFrame({'id': list(range(1, len(lis)+1)), 'label': lis})
submissionAssignment8_2.to_csv('submissionAssignment8_2.csv', index=False)

###Conclusion

If I were advising management on classifying customer reviews, I believe doing RNN models would be the best way to go. This way, every word is taken into account. One thing we have to do effectively is preprocesing. Something I missed this assignment was removing the extremely common words, such as 'like', 'the', 'a', etc.. Those may waste the models time and may even cause inaccuracies in predictions. Instead a lot of its energy should be focused on identifying keywords that have a lot of meaning, such as 'bad', 'serious', 'terrible', 'hate', etc.. These words are what will have the most impact in identifying what kind of review was left. Data cleaning and preprocessing is more important than creating the model itself at times, as the model is only as good as the data fit into it. It may be better to even train it on a lot of data and some anomalies.On top of that, how effectively you train the model with good parameters also has great effects on how the model performs. You have to let it train a good amount of times for it to find differences, but also not too much which may lead to overfitting.